<a href="https://colab.research.google.com/github/NisharaniMeti/ENI/blob/main/Multi_label_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
#!pip install simpletransformers

In [ ]:
#load dataset
df = pd.read_csv('//content/train.csv')
df.head()

In [ ]:
df['labels'] = list(zip(df.toxic.tolist(), df.severe_toxic.tolist(), df.obscene.tolist(), df.threat.tolist(),  df.insult.tolist(), df.identity_hate.tolist()))
#df['labels'] = df['labels'].astype(int)
df['text'] = df['comment_text'].apply(lambda x: str(x).replace('\n', ' '))

df.head()

In [ ]:
#split the df into train and eval datasets so we can validate the model
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df, test_size=0.2)

In [ ]:
#Create Multi-label Classification Model
from simpletransformers.classification import MultiLabelClassificationModel


model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=6, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

In [ ]:
# Train the model
model.train_model(train_df)

In [ ]:
#Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:
#Predict/test the model
test_df = pd.read_csv('//content/test.csv')

to_predict = test_df.comment_text.apply(lambda x: x.replace('\n', ' ')).tolist()
preds, outputs = model.predict(to_predict)

sub_df = pd.DataFrame(outputs, columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate'])


sub_df['id'] = test_df['id']
sub_df = sub_df[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate']]

sub_df.to_csv('//content/outputs/submission.csv', index=False)